In [1]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def cv_show(name,img):
    cv2.imshow(name,img,)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_pc(file_path, number):
    #number= 0
    #numb = str(number)
    img = cv2.imread(file_path)
    #图像灰度图
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #自适应均衡化
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(6,6)) 
    img_gray = clahe.apply(img_gray)
    #cv_show('img_gray',img_gray)
    #高斯滤波去噪
    #img_gray_Gau = median = cv2.medianBlur(img_gray, 5)  
    img_gray_Gau = cv2.GaussianBlur(img_gray, (3, 3), 1) 
#     cv2.imshow('img_gray_Gau',img_gray_Gau)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # 二值图像
    #ref, thresh = cv2.threshold(img_gray_Gau,90, 255, cv2.THRESH_BINARY)#THRESH_BINARY的反转
    thresh = cv2.threshold(img_gray_Gau, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1] 
#     cv_show('thresh',thresh)

    #梯度运算
    # 梯度=膨胀-腐蚀
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)
    kernel = np.ones((8,8),np.uint8) 
    gradient = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)

#     cv2.imshow('gradient', gradient)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # 初始化卷积核
    Kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    Kernel2 = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))
    Kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    #Kernel4 = np.zeros((6,6),np.uint8)

    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,Kernel2) 
#     cv_show('thresh',thresh)

    #thresh = cv2.erode(thresh,Kernel1,iterations = 4)

    Kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE,Kernel1) 
#     cv_show('thresh',thresh)

    #thresh = cv2.erode(thresh,Kernel1,iterations = 4)

    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    thresh = cv2.erode(thresh,kernel1,iterations = 4)
#     cv_show('thresh',thresh)

    #使用canny边缘提取算法提取边缘特征
    v1 = cv2.Canny(thresh, 80, 130)
    v2 = cv2.Canny(thresh, 70, 120)

    thresh = np.hstack((v1,v2))
#     cv_show('thresh',thresh)

    # 计算轮廓
    #cv2.findContours()
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    draw_img = img.copy()
    res = cv2.drawContours(draw_img,contours,-1, (0,0,255) , 2 ) 
    cv2.imwrite(f"./result/{number}"  + ".png", res)
    #cv_show('res',res)

    cnt = contours.copy()
    list_loc = []
    location = {}
    (x_b, y_b, z_b) = img.shape
    for (i , c) in enumerate(cnt):
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w/float(h)
        if ar > 0.8 and ar< 1.2:
            x1 = c[0][0][0]
            y1 = c[0][0][1]
            x2 = c[-1][0][0]
            y2 = c[-1][0][1]
            if abs(x1-x2)<=1 and abs(y1-y2)<=1:
                if x< x_b and y<y_b:
                    if 260<=cv2.contourArea(c) and cv2.contourArea(c)<=2000:
                        location[x, y, w, h]= i
    wid = 0   
    index = 0
    for key, v in location.items():
        if wid <list(key)[3]:
            wid = list(key)[3]
            index = v


    draw_img = img.copy()
    res = cv2.drawContours(draw_img,contours[index],-1, (0,0,255) , 2 ) 
    cv2.imwrite(f"./result/{number}" + ".jpg",res)
    #cv_show('res',res)
    #number = number +1
    

In [3]:
import os
number = 0
for file_name in os.listdir('./LED'):
    try:
        get_pc(f"./LED/{file_name}", number)
        number =  number + 1
    except Exception as e:
        print(file_name)